In [1]:
import preprocess

In [2]:
tst = [2023, 8, 28, 'AM']
ted = [2023, 9, 6, 'AM']
region = ('CONUS', [-126.401171875,24.071240929282325,
                    -61.36210937500001,49.40003415463647])

In [3]:
import FireTime

list_of_ts = list(FireTime.t_generator(tst, ted))
list_of_ts

[[2023, 8, 28, 'AM'],
 [2023, 8, 28, 'PM'],
 [2023, 8, 29, 'AM'],
 [2023, 8, 29, 'PM'],
 [2023, 8, 30, 'AM'],
 [2023, 8, 30, 'PM'],
 [2023, 8, 31, 'AM'],
 [2023, 8, 31, 'PM'],
 [2023, 9, 1, 'AM'],
 [2023, 9, 1, 'PM'],
 [2023, 9, 2, 'AM'],
 [2023, 9, 2, 'PM'],
 [2023, 9, 3, 'AM'],
 [2023, 9, 3, 'PM'],
 [2023, 9, 4, 'AM'],
 [2023, 9, 4, 'PM'],
 [2023, 9, 5, 'AM'],
 [2023, 9, 5, 'PM'],
 [2023, 9, 6, 'AM']]

## Once per landcover file

If your landcover file is not in lat,lon then transform it and resave it. You will use this file a bunch later on.

In [4]:
preprocess.preprocess_landcover()

2024-03-20 19:30:16,186 - FireLog - INFO - func:preprocess_landcover took: 22.65 sec


## Once per region

Preprocess the region to get rid of static flare sources. Save that new "swiss cheese" shape off into a geojson file for later.

In [5]:
preprocess.preprocess_region(region)

2024-03-20 19:32:47,689 - FireLog - INFO - func:preprocess_region took: 2.05 min


## Once per input file

Next process each NRT file into half day files. First we'll get all the times that are of interest. This could also be done by inspecting all looking at all the files that exist and seeing which have not been preprocessed yet.

In [6]:
%%time
for sat in ["SNPP", "NOAA20"]:
    for t in list_of_ts[::2]:
        preprocess.preprocess_NRT_file(t, sat)

2024-03-20 19:33:16,131 - FireLog - INFO - preprocessing NRT file for 2023-8-28, SNPP
/home/jovyan/fireatlas_nrt/FireIO.py:312: FutureWarning: Support for nested sequences for 'parse_dates' in pd.read_csv is deprecated. Combine the desired columns with pd.to_datetime after parsing instead.
  df = pd.read_csv(
2024-03-20 19:33:17,763 - FireLog - INFO - func:preprocess_NRT_file took: 1.63 sec
2024-03-20 19:33:17,764 - FireLog - INFO - preprocessing NRT file for 2023-8-29, SNPP
/home/jovyan/fireatlas_nrt/FireIO.py:312: FutureWarning: Support for nested sequences for 'parse_dates' in pd.read_csv is deprecated. Combine the desired columns with pd.to_datetime after parsing instead.
  df = pd.read_csv(
2024-03-20 19:33:19,140 - FireLog - INFO - func:preprocess_NRT_file took: 1.38 sec
2024-03-20 19:33:19,141 - FireLog - INFO - preprocessing NRT file for 2023-8-30, SNPP
/home/jovyan/fireatlas_nrt/FireIO.py:312: FutureWarning: Support for nested sequences for 'parse_dates' in pd.read_csv is depr

CPU times: user 14 s, sys: 276 ms, total: 14.3 s
Wall time: 27.1 s


## Once per region and t

Do initial filtering and clustering using the preprocessed region and the half day files.

Note: for the purpose of timing I am running the full month, but each of these steps could run in a separate proccess.

In [7]:
%%time
for t in list_of_ts:
    preprocess.preprocess_region_t(t, sensor="VIIRS", region=region)

2024-03-20 19:33:49,640 - FireLog - INFO - filtering and clustering 2023-8-28 AM, VIIRS, CONUS
2024-03-20 19:33:49,693 - FireLog - INFO - func:read_preprocessed took: 51.71 ms
2024-03-20 19:33:49,738 - FireLog - INFO - func:read_preprocessed took: 44.49 ms
2024-03-20 19:33:50,112 - FireLog - INFO - func:read_region took: 371.37 ms
2024-03-20 19:35:51,591 - FireLog - INFO - func:do_clustering took: 502.84 ms
2024-03-20 19:35:51,665 - FireLog - INFO - func:preprocess_region_t took: 2.03 min
2024-03-20 19:35:51,666 - FireLog - INFO - filtering and clustering 2023-8-28 PM, VIIRS, CONUS
2024-03-20 19:35:51,735 - FireLog - INFO - func:read_preprocessed took: 68.20 ms
2024-03-20 19:35:51,799 - FireLog - INFO - func:read_preprocessed took: 63.15 ms
2024-03-20 19:35:52,106 - FireLog - INFO - func:read_region took: 303.99 ms
2024-03-20 19:37:03,239 - FireLog - INFO - func:do_clustering took: 28.65 ms
2024-03-20 19:37:03,296 - FireLog - INFO - func:preprocess_region_t took: 1.19 min
2024-03-20 19

CPU times: user 18min 41s, sys: 468 ms, total: 18min 41s
Wall time: 19min 2s
